# 120 Interpolating observation positions / merge

In [ ]:
load_from_config = True
raw_data_folder = ""
preprocessed_data_folder = ""

In [ ]:
%matplotlib widget 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import yaml

In [ ]:
with open('../config.yaml') as file:
  params= yaml.safe_load(file)

# in case of a automatic run
if not load_from_config:
    params['raw_data_folder'] = raw_data_folder
    params['preprocessed_data_folder'] = preprocessed_data_folder

preprocessed_data_folder = params['data_folder'] + params['preprocessed_data_folder']


In [ ]:
df_humans = pd.read_pickle(preprocessed_data_folder + 'pp_humans.pkl')
df_humans['timestamp_gnss'] = pd.to_datetime(df_humans['gnss.timestamp'], unit='ns')
print(f"{df_humans.shape[0]} entries in the human dataset")
print(f"{df_humans.timestamp_gnss.min()} - {df_humans.timestamp_gnss.max()}")

In [ ]:
df_robot = pd.read_pickle(preprocessed_data_folder + "pp_robot.pkl")
print(f"{df_robot.shape[0]} entries in the robot dataset")
print(f"{df_robot.timestamp.min()} - {df_robot.timestamp.max()}")

In [ ]:
df_robot.head(3).T

## Approximate the robot's position according to human's timestamp

In [ ]:
def find_neighbored_positions(timestamp, df):
    timestamp_column = 'timestamp'
    dist = (df[timestamp_column] - timestamp).dt.total_seconds()
    positive_dist = dist[dist > 0]
    negative_dist = dist[dist < 0]
    if positive_dist.empty and ~negative_dist.empty:
        before = df.loc[negative_dist.idxmax(), timestamp_column]
        after = None
    elif ~positive_dist.empty and negative_dist.empty:
        before = None
        after = df.loc[positive_dist.idxmin(), timestamp_column]
    else:
        before = df.loc[negative_dist.idxmax(), timestamp_column]
        after = df.loc[positive_dist.idxmin(), timestamp_column]
    return before, after

print(df_humans.iloc[0]['obj.timestamp'])
find_neighbored_positions(df_humans.iloc[0]['obj.timestamp'], df_robot)

In [ ]:
for index, row in df_humans.iterrows():
    before, after = find_neighbored_positions(row['obj.timestamp'], df_robot)
    df_humans.at[index, 'robot_before'] = before
    df_humans.at[index, 'robot_after'] = after
    if before is not None and after is not None and (after - before) < pd.Timedelta('2s'):
        df_humans.at[index, 'valid_interpolation'] = True
    else:
        df_humans.at[index, 'valid_interpolation'] = False

#df_humans['valid_interpolation'] = (df_humans['robot_before'] - df_humans['robot_after']) < pd.Timedelta('1s')


In [ ]:
df_humans[['obj.timestamp', 'robot_before', 'robot_after', 'valid_interpolation']].tail(5)

In [ ]:
df_humans.valid_interpolation.value_counts()

In [ ]:
df_robot.loc[df_robot['timestamp'] == df_humans.iloc[0]['robot_before']]

In [ ]:
for index, row in df_humans.iterrows():
    if row['valid_interpolation']:
        
        before = df_robot[df_robot['timestamp'] == row['robot_before']].iloc[0]
        after = df_robot[df_robot['timestamp'] == row['robot_after']].iloc[0]
        ratio_time_diff = (row['obj.timestamp'] - row['robot_before']) / (row['robot_after'] - row['robot_before'])

        df_humans.at[index, 'robot_latitude'] = ratio_time_diff*(after.latitude - before.latitude) + before.latitude
        df_humans.at[index, 'robot_longitude'] = ratio_time_diff*(after.longitude - before.longitude) + before.longitude
        diff_bearing = after.bearing - before.bearing
        if diff_bearing > np.pi:
            diff_bearing = diff_bearing - 2* np.pi
        if diff_bearing < -np.pi:
            diff_bearing = diff_bearing + 2* np.pi
        df_humans.at[index, 'bearing'] = ratio_time_diff*diff_bearing + before.bearing
        
        # for testing purposes
        df_humans.at[index, 'robot_latitude_before'] = before.latitude
        df_humans.at[index, 'robot_longitude_before'] = before.longitude
        df_humans.at[index, 'robot_latitude_after'] = after.latitude
        df_humans.at[index, 'robot_longitude_after'] = after.longitude

        # for evaluation of positional information
        df_humans.at[index, 'pose_valid'] = before.pose_valid

        df_humans.at[index, 'speed'] = ratio_time_diff*(after.speed - before.speed) + before.speed
        df_humans.at[index, 'heading_diff'] = ratio_time_diff*(after.heading_diff - before.heading_diff) + before.heading_diff
        df_humans.at[index, 'HDOP'] = ratio_time_diff*(after.HDOP - before.HDOP) + before.HDOP

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
df_humans.plot.scatter(x='robot_longitude', y='robot_latitude', color='red', s=0.1, ax=ax)
df_humans.plot.scatter(x='robot_longitude_before', y='robot_latitude_before', s=0.1, ax=ax)

# Zoom into the map for checking the interpolation

In [ ]:
preprocessed_data_folder = params['data_folder'] + params['preprocessed_data_folder']
pd.to_pickle(df_humans, preprocessed_data_folder + params['preprocessed_file_labels']+ "merged.pkl")